# Esimerkki Näyte Hotelli- ja Lentovarausagentti

Tämä ratkaisu auttaa sinua varaamaan lentoliput ja hotellin. Tilanne on matka Lontoon Heathrow'lta (LHR) 20. helmikuuta 2024 New Yorkin JFK:lle, paluu 27. helmikuuta 2025, lentäen turistiluokassa vain British Airwaysilla. Haluan yöpyä Hilton-hotellissa New Yorkissa, ole hyvä ja anna kustannukset lennolle ja hotellille.


# Alusta Azure AI Agent Service ja hanki konfiguraatiotiedot **.env**-tiedostosta

### **.env**

Luo .env-tiedosto

**.env** sisältää Azure AI Agent Servicen yhteysmerkkijonon, AOAI:n käyttämän mallin sekä vastaavan Google API Search -palvelun API:n, ENDPOINT:in jne.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Azure AI Agent Servicen mallin käyttöönoton nimi"

[**NOTE**] Tarvitset mallin, jonka rajoitus on 100 000 (tokenia minuutissa) ja 600 (pyyntöä minuutissa).

  Mallin voi hankkia Azure AI Foundry -sivustolta kohdasta Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Azure AI Agent Servicen projektin yhteysmerkkijono"

  Projektin yhteysmerkkijonon löydät projektin yleiskatsauksesta AI Foundry -portaalin näytöltä.

- **SERPAPI_SEARCH_API_KEY** = "SERPAPI-hakupalvelun API-AVAIMESI"
- **SERPAPI_SEARCH_ENDPOINT** = "SERPAPI-hakupalvelun ENDPOINT"

Azure AI Agent Servicen mallin käyttöönoton nimen ja projektin yhteysmerkkijonon hankkimiseksi sinun täytyy luoda Azure AI Agent Service. Suositellaan käyttämään [tätä mallia](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) sen luomiseen suoraan (***Huom:*** Azure AI Agent Service on tällä hetkellä saatavilla vain rajoitetuilla alueilla. Suositellaan viittaamaan [tähän linkkiin](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) alueen asettamiseksi).

Agentin täytyy päästä käsiksi SERPAPI:in. Suositellaan rekisteröitymistä [tämän linkin](https://serpapi.com/searches) kautta. Rekisteröitymisen jälkeen voit hankkia yksilöllisen API-AVAIMEN ja ENDPOINT:in.


# Asennus

Tämän muistikirjan suorittamiseksi sinun tulee varmistaa, että olet asentanut tarvittavat kirjastot suorittamalla `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Sinun Semantic Kernel -version tulee olla vähintään 1.27.2.


Lataa .env-tiedostosi asetukset ja resurssit, varmista, että olet lisännyt avaimet ja asetukset sekä luonut paikallisen .env-tiedoston.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Kirjaudu sisään Azureen

Sinun täytyy nyt kirjautua sisään Azureen. Avaa pääte ja suorita seuraava komento:

```bash
az login
```

Tämä komento pyytää sinua syöttämään Azuren tunnistetietosi, jotta Azure AI Agent -palvelu voi toimia oikein.


# Selitys:
Tämä on muuttuja, joka tallentaa API-avaimen SERP (Search Engine Results Page) API-palvelun käyttöä varten. API-avain on yksilöllinen tunniste, jota käytetään tilisi kanssa yhdistettyjen pyyntöjen todentamiseen.

Tarkoitus: Tämän rivin tarkoitus on tallentaa API-avain muuttujaan, jotta sitä voidaan käyttää pyyntöjen todentamiseen SERP API -palvelussa. API-avain on välttämätön palvelun käyttämiseksi ja hakujen suorittamiseksi.
Kuinka hankkia SERP API -avain: Hankkiaksesi SERP API -avaimen, noudata näitä yleisiä ohjeita osoitteessa https://serpapi.com (tarkat vaiheet voivat vaihdella käyttämäsi SERP API -palvelun mukaan):

Valitse SERP API -palvelu: Saatavilla on useita SERP API -palveluita, kuten SerpAPI, Google Custom Search JSON API ja muita. Valitse palvelu, joka parhaiten vastaa tarpeitasi.

Rekisteröidy tilille: Siirry valitsemasi SERP API -palvelun verkkosivustolle ja rekisteröidy tilille. Sinun tulee mahdollisesti antaa perustietoja ja vahvistaa sähköpostiosoitteesi.

Luo API-avain: Rekisteröitymisen jälkeen kirjaudu tilillesi ja siirry API-osioon tai hallintapaneeliin. Etsi vaihtoehto uuden API-avaimen luomiseen tai generointiin.
Kopioi API-avain .env-tiedostoosi.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Selitys:
BASE_URL: Tämä on muuttuja, joka tallentaa SERP API -päätepisteen perus-URL-osoitteen. Muuttujan nimi BASE_URL on konventio, joka osoittaa, että tämä URL on lähtökohta API-pyyntöjen tekemiselle.
'https://serpapi.com/search':

Tämä on varsinainen URL-merkkijono, joka on määritetty BASE_URL-muuttujalle. Se edustaa päätepistettä, jota käytetään hakukyselyiden suorittamiseen SERP API:n avulla.

# Tarkoitus:
Tämän rivin tarkoitus on määritellä vakio, joka sisältää SERP API:n perus-URL-osoitteen. Tätä URL-osoitetta käytetään lähtökohtana API-pyyntöjen rakentamisessa hakutoimintojen suorittamiseksi.

# Käyttö:
Määrittelemällä perus-URL muuttujassa voit helposti käyttää sitä uudelleen koodissasi aina, kun sinun tarvitsee tehdä pyyntöjä SERP API:lle. Tämä tekee koodistasi helpommin ylläpidettävän ja vähentää virheiden riskiä, jotka johtuvat URL-osoitteen kovakoodaamisesta useisiin paikkoihin. Nykyinen esimerkki on https://serpapi.com/search?engine=bing, joka käyttää Bing-hakukoneen API:a. Eri API voidaan valita osoitteessa https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Selitys:

Tässä on paikka, jossa liitännäiskoodisi sijaitsee.

Luokan määrittely: `class BookingPlugin`: Määrittelee luokan nimeltä BookingPlugin, joka sisältää menetelmiä hotellien ja lentojen varaamiseen.

Hotellivarauksen menetelmä:

- `@kernel_function(description="booking hotel")`: Koristaja, joka kuvaa toiminnon ytimenä hotellivarauksia varten.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Määrittelee menetelmän hotellivarauksille, jossa on annotoidut parametrit ja palautustyyppi.

Menetelmä rakentaa sanakirjan hotellivarauspyynnön parametreista ja lähettää GET-pyynnön SERP API:lle. Se tarkistaa vastauksen tilan ja palauttaa hotellin tiedot, jos pyyntö onnistuu, tai None, jos pyyntö epäonnistuu.

Lentovarauksen menetelmä:

- `@kernel_function(description="booking flight")`: Koristaja, joka kuvaa toiminnon ytimenä lentovarauksia varten.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Määrittelee menetelmän lentovarauksille, jossa on annotoidut parametrit ja palautustyyppi.

Menetelmä rakentaa sanakirjat meno- ja paluulentopyyntöjen parametreista ja lähettää GET-pyynnöt SERP API:lle. Se tarkistaa vastauksen tilan ja lisää lentotiedot tulosmerkkijonoon, jos pyyntö onnistuu, tai tulostaa virheilmoituksen, jos pyyntö epäonnistuu. Menetelmä palauttaa tulosmerkkijonon, joka sisältää lentotiedot.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Selitys:
Tuontilauseet: Tuo tarvittavat moduulit Azure-tunnistetietoja, AI-agenttia, chat-viestin sisältöä, kirjoittajan roolia ja kernel-funktion koristetta varten.

Asynkroninen kontekstinhallinta: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Tämä asettaa asynkronisen kontekstinhallinnan käsittelemään Azure-tunnistetietoja ja luomaan AI-agentin asiakasohjelman.

Agentin nimi ja ohjeet: 
- `AGENT_NAME = "BookingAgent"`: Määrittää agentin nimen.
- `AGENT_INSTRUCTIONS = """..."""`: Tarjoaa yksityiskohtaiset ohjeet agentille varausten käsittelyä varten.

Luo agenttimääritelmä: `agent_definition = await client.agents.create_agent(...)`: Luo agenttimääritelmän määritetyllä mallilla, nimellä ja ohjeilla.

Luo AzureAI-agentti: `agent = AzureAIAgent(...)`: Luo AzureAI-agentin käyttäen asiakasohjelmaa, agenttimääritelmää ja määriteltyä liitännäistä.

Luo säie: `thread: AzureAIAgentThread | None = None`: Luo säie agentille. Säiettä ei tarvitse luoda ensin - jos arvoksi annetaan `None`, uusi säie luodaan ensimmäisen kutsun aikana ja palautetaan osana vastausta.

Käyttäjän syötteet: `user_inputs = ["..."]`: Määrittää listan käyttäjän syötteistä, jotka agentti käsittelee.

Lopuksi-lohkossa poista säie ja agentti vapauttaaksesi resurssit.


# Autentikointi

`DefaultAzureCredential`-luokka on osa Azure SDK:ta Pythonille. Se tarjoaa oletustavan autentikoida Azure-palveluiden kanssa. Luokka yrittää autentikoida useilla menetelmillä tietyssä järjestyksessä, kuten ympäristömuuttujilla, hallinnoidulla identiteetillä ja Azure CLI -tunnuksilla.

Asynkroniset toiminnot: aio-moduuli osoittaa, että DefaultAzureCredential-luokka tukee asynkronisia toimintoja. Tämä tarkoittaa, että voit käyttää sitä asyncio:n kanssa suorittaaksesi ei-estäviä autentikointipyyntöjä.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
